In [1]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
# from datatools.analyzer import clean_text
from error_tools import *

In [2]:
from nltk.lm        import Vocabulary
# from nltk.lm.models import MLE
from nltk.lm.models import KneserNeyInterpolated
from nltk.util      import ngrams

In [3]:
def read_file(filename, batch_size=10000) -> list:
    f = open(filename, 'r', encoding='UTF-8')
    text_list = [  ]
    concated = ""
    for i, line in enumerate( f.readlines() ):
        concated += line.rstrip('\n')
        if (i+1) % batch_size == 0:
            text_list.append(concated)
            concated = ""
    text_list.append(concated)
    f.close()
    return text_list

filename = "../../corpus/wiki/wiki_40b_train_normal.txt"
all_list = read_file(filename, batch_size=100)
print("len of text_list :", len(all_list))


len of text_list : 18674


In [83]:
all_wiki = []
for line in tqdm(all_list):
    all_wiki += [sen+"。" for sen in line.split("。")]

100%|██████████| 18674/18674 [00:03<00:00, 5287.78it/s]


In [84]:
wiki_utt = all_wiki[::100]

In [85]:
len(wiki_utt)

123490

In [42]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)
conv_utt = []
for conv in convs:
    for i, ut in enumerate(conv):
        if not ut.is_exist_error():
            conv_utt.append(clean_text(ut.utt))

In [41]:
from tqdm import tqdm
def extract_utt_nucc(path):
    files = os.listdir(path)
    nucc_convs = []
    for filename in tqdm(files):
        if ".json" not in filename:
            continue
        # name = filename.split(".")[0]
        with open(path+filename, "r") as f:
            data  = json.load(f)
            for conv in data["turns"]:
                utt = conv["utterance"]
                if len(nlp(utt)) < 2:
                    # print(utt)
                    continue
                nucc_convs.append(clean_text(utt))
    return nucc_convs
# nuccデータ
nucc_path = "../../corpus/nucc/conv2/"
nucc_convs = extract_utt_nucc(nucc_path)

100%|██████████| 91/91 [09:19<00:00,  6.15s/it]


In [43]:
def load_utt_ntt():
    ntt_path = "../../corpus/NTT/"
    utt_list = []
    for file_ in os.listdir(ntt_path):
        if not "json" in file_:
            continue 
        with open(ntt_path+file_, "r",  encoding="utf-8") as f:
            convs = json.load(f)
            for did in convs["convs"]:
                dids = list( did.keys() )[0]
                conv = did[dids]
                # conv = did[dids][3::3]
                utt_list.extend( [ utt for utt in conv])
    
    print(len(utt_list))
    return utt_list


In [46]:
ntt_utt = load_utt_ntt()

141777


In [127]:
corpus = conv_utt + nucc_convs + ntt_utt + all_wiki

In [128]:
len(corpus)

12536965

In [88]:
def clean_text_plain(text):
    text_ = neologdn.normalize(text)
    text_ = re.sub(r'\(.*\)', "", text_)
    text_ = re.sub(r'\d+', "0", text_)
    if "？？" in text_:
        text_ = text_.replace("？？", "？")
    return text_

In [129]:
all_utt = [clean_text_plain(t) for t in tqdm(corpus)]

100%|██████████| 12536965/12536965 [03:56<00:00, 53042.77it/s]


In [111]:
mecab_analyzer = Analyzer(tokenizer_)
mecab_analyzer = Analyzer(Tokenizer())

def last_pos(pos_split, filler="*"):
    last = len(pos_split) - 1 
    for i, _ in enumerate(pos_split):
        if pos_split[last-i] != filler :
            return "["+pos_split[last-i]+"]"
def sentence2normalize_noun_mecab(sentences):
    normalize_sen = []
    for sen in tqdm(sentences):
        df = mecab_analyzer.analyze_with_dataframe(sen)
        words = []
        if df is None:
            continue
        for txt, pos in zip(df.surface, df.part_of_speech):
            pos_split = pos.split("-")
            # if pos_split[0]
            # if pos_split[1] != "サ変接続" and pos_split[1] != "非自立" and pos_split[1] != "接尾" and "名詞" == pos_split[0]:
            if pos_split[0]=="名詞":
                # words.append(pos)
                # print(txt, pos)
                last_pos_name = last_pos(pos_split, filler="")
                if last_pos_name=="[サ変可能]":
                    words.append(txt)
                else:
                    words.append(last_pos_name)
            else:
                words.append(txt)
        normalize_sen.append(words)
    return normalize_sen

In [112]:
filled_normal = fill_SYMBOL_ONE( sentence2normalize_noun_mecab(all_utt[-30:]) )

100%|██████████| 30/30 [00:00<00:00, 729.27it/s]


In [114]:
# filled_normal

In [115]:
normal = sentence2normalize_noun_mecab(all_utt)

100%|██████████| 311503/311503 [05:15<00:00, 988.74it/s] 


In [116]:
normal_split = [ list("".join(L)) for L in tqdm(normal) ]

100%|██████████| 311502/311502 [00:03<00:00, 96050.40it/s] 


In [130]:
char_split = [ list("".join(L)) for L in tqdm(all_utt) ]

 40%|███▉      | 4964752/12536965 [00:49<00:39, 192488.33it/s]

In [117]:
def create_language_model(sentences, N):
    vocab = Vocabulary([word for sent in sentences for word in sent])
    text_ngrams = [ngrams(sent, N) for sent in sentences]
    lm = KneserNeyInterpolated(order=N, vocabulary=vocab)
    lm.fit(text_ngrams)
    return lm

In [122]:
n=4
lm = create_language_model(char_split, N=n)

In [123]:
from datatools.maneger import DataManager
modelM = DataManager("../models/utterance/")
model_name = "KLM_phrase_nucc_ntt_wiki_n={0}_splited_char.pickle".format(n)
# model_name = "KLM_phrase_nucc_wiki_n={0}_noun2.pickle".format(n)

In [124]:
modelM.save_data(model_name, lm)

success save : ../models/utterance/KLM_phrase_nucc_ntt_wiki_n=4_splited_unidic.pickle


In [121]:
# dataM = DataManager("../X_y_data/utterance/")
# dataM.save_data("nucc_ntt_split_char.pickle", normal_split)